# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [2]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]

# Use the relative path from the 'assignments' directory
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header=None, names=columns)
              .assign(income=lambda x: (x.income.str.strip() == '>50K') * 1))



# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [3]:

from sklearn.model_selection import train_test_split


# Separate features and target
X = adult_dt.drop('income', axis=1)  # All columns except income
Y = adult_dt['income']  # Only the income column

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Check the shapes to confirm
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)


X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792,)
Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

The random state in a data splitting function (like train_test_split in scikit-learn) is a parameter that controls the randomness of the data split. Every time you run the code, with the same data and the same random state, you get the exact same split into training and test sets.

It is useful for:
Reproducibility: It ensures that anyone running the code gets the same results, especially, when you’re collaborating, sharing results, or debugging.

Consistency in Model Evaluation: When testing different models or model parameters, it evaluates each model on the same split of training and testing data.
By setting random state, one ensures that all models are evaluated on the exact same test set, making comparisons fair and consistent.

Easier Experimentation: With a fixed random state, one can try different algorithms, parameters, or feature sets without worrying about differences in training/testing splits.


# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [5]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Define column names
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]

# Load the data
adult_dt = pd.read_csv('../../05_src/data/adult/adult.data', header=None, names=columns)
adult_dt['income'] = adult_dt['income'].str.strip()

# Separate features and target
X = adult_dt.drop('income', axis=1)
Y = adult_dt['income']

# Define pipelines
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Pipeline for numerical features
numerical_pipeline = Pipeline(steps=[
    ('knn_imputer', KNNImputer(n_neighbors=7, weights="distance")),
    ('scaler', RobustScaler())
])

# Pipeline for categorical features
categorical_pipeline = Pipeline(steps=[
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Column Transformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, numerical_features),
    ('cat', categorical_pipeline, categorical_features)
])

# Apply transformations
X_transformed = preprocessor.fit_transform(X)

# Print shape to confirm
print("Transformed X shape:", X_transformed.shape)


Transformed X shape: (32561, 107)


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [6]:
# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Define the model pipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # Column Transformer for preprocessing
    ('classifier', RandomForestClassifier())  # RandomForest classifier
])

# Fit the pipeline on the training data
model_pipeline.fit(X_train, Y_train)

# Predict on the test data
predictions = model_pipeline.predict(X_test)

# Optional: Print sample predictions or evaluate model performance
print("Sample Predictions:", predictions[:10])


Sample Predictions: ['<=50K' '<=50K' '>50K' '<=50K' '<=50K' '>50K' '>50K' '<=50K' '<=50K'
 '>50K']


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [7]:
from sklearn.model_selection import cross_validate
import pandas as pd

# Define the scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',       # Negative log loss
    'roc_auc': 'roc_auc',                 # ROC AUC
    'accuracy': 'accuracy',               # Accuracy
    'balanced_accuracy': 'balanced_accuracy'  # Balanced accuracy
}

# Perform cross-validation with 5 folds
cv_results = cross_validate(
    model_pipeline, 
    X_train, Y_train, 
    cv=5, 
    scoring=scoring,
    return_train_score=True
)

# Convert cross-validation results to a DataFrame for easier interpretation
cv_results_df = pd.DataFrame(cv_results)

# Display the fold-level results for training and validation metrics
print("Fold-Level Cross-Validation Results:")
print(cv_results_df)


c:\Users\Admin\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Admin\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Fold-Level Cross-Validation Results:
    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
0  25.782416    0.721329          -0.349779           -0.081280      0.904580   
1  26.025859    0.605199          -0.361224           -0.081217      0.902057   
2  27.426582    0.616514          -0.376189           -0.081322      0.900801   
3  25.348829    0.582020          -0.357874           -0.082438      0.906503   
4  26.431999    0.766695          -0.360224           -0.081852      0.902474   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
0            1.0       0.849528        0.999890                0.772531   
1            1.0       0.846896        1.000000                0.767414   
2            1.0       0.851689        0.999945                0.773809   
3            1.0       0.861562        1.000000                0.785248   
4            1.0       0.853006        1.000000                0.774677   

   train_balanced_accurac

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [8]:

# Assume `cv_results` is the output from `cross_validate` function
# Convert the cross-validation results to a DataFrame
cv_results_df = pd.DataFrame(cv_results)

# Sort the DataFrame by 'test_neg_log_loss' in descending order
# (if we want higher values of negative log loss to be at the top)
cv_results_df_sorted = cv_results_df.sort_values(by='test_neg_log_loss', ascending=False)

# Display the sorted DataFrame
print("Fold-Level Cross-Validation Results Sorted by Test Negative Log Loss:")
print(cv_results_df_sorted)


Fold-Level Cross-Validation Results Sorted by Test Negative Log Loss:
    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
0  25.782416    0.721329          -0.349779           -0.081280      0.904580   
3  25.348829    0.582020          -0.357874           -0.082438      0.906503   
4  26.431999    0.766695          -0.360224           -0.081852      0.902474   
1  26.025859    0.605199          -0.361224           -0.081217      0.902057   
2  27.426582    0.616514          -0.376189           -0.081322      0.900801   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
0            1.0       0.849528        0.999890                0.772531   
3            1.0       0.861562        1.000000                0.785248   
4            1.0       0.853006        1.000000                0.774677   
1            1.0       0.846896        1.000000                0.767414   
2            1.0       0.851689        0.999945                0.773

Calculate the mean of each metric. 

In [9]:
# Calculate the mean of each metric across folds
mean_results = cv_results_df.mean()

# Display the mean results
print("Mean Cross-Validation Results:")
print(mean_results)


Mean Cross-Validation Results:
fit_time                   26.203137
score_time                  0.658351
test_neg_log_loss          -0.361058
train_neg_log_loss         -0.081622
test_roc_auc                0.903283
train_roc_auc               1.000000
test_accuracy               0.852536
train_accuracy              0.999967
test_balanced_accuracy      0.774736
train_balanced_accuracy     0.999932
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [10]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Get predicted probabilities and class predictions for X_test
Y_pred_proba = model_pipeline.predict_proba(X_test)  # Probability predictions for each class
Y_pred = model_pipeline.predict(X_test)              # Class label predictions

# Calculate each metric on the test set
test_metrics = {
    'neg_log_loss': -log_loss(Y_test, Y_pred_proba),               # Negative log loss
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),          # ROC AUC (using only positive class probabilities)
    'accuracy': accuracy_score(Y_test, Y_pred),                    # Accuracy
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)   # Balanced accuracy
}

# Display the results as a dictionary
print("Test Set Performance Metrics:")
print(test_metrics)


Test Set Performance Metrics:
{'neg_log_loss': -0.3764444654687329, 'roc_auc': 0.9006139400505597, 'accuracy': 0.8504452861091207, 'balanced_accuracy': 0.7674587716445606}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

Recoding the target variable income immediately after loading the data is convenient for several reasons:

1. Standardizes the Target Variable for Modeling

2. Simplifies Later Processing and Analysis

3. Improves Readability and Reduces Code Duplication


## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.